In [10]:
#import pynq libraries
from pynq import Overlay
from pynq import allocate
import math
import time
import numpy as np

In [11]:
#show documentation
#testmem_ip?

In [12]:
#show testmem ip register maps
#testmem_ip.register_map

In [8]:
Mbyte=80
for n in range(3):
    #kernel function's IP made for write from the FPGA
    if n==0:
        overlay =Overlay("./design_64_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=64
        w_time_64= np.zeros(20)
    if n==1:
        overlay =Overlay("./design_1_128_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=128
        w_time_128=np.zeros(20)
    if n==2:
        overlay =Overlay("./design_1_256_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=256
        w_time_256=np.zeros(20)
    
    #inizialize data
    max_data=int((Mbyte*1000*1000*8)/datatype)
    #FPGA
    data= np.zeros(int(max_data*datatype/64))
    #data buffer creation for FPGA
    buf_data = allocate(int(max_data*datatype/64),np.uint64)
    buf_data[:]=data[:]
    buf_data.flush()
    testmem_ip.write(0x10,buf_data.physical_address)
    
    #kernel function attivation and time calculation
    
    #decide Mbyte intervals 
    sud=int(Mbyte/20)
    data_n=int((sud*1000*1000*8)/datatype)
    #starting write time calculation
    
    for p in range(20):
        p=1+p
        #100 subtests to create an avarage time
        w_sum_64=0
        w_sum_128=0
        w_sum_256=0
        for c in range(100):
            data_n=int((sud*1000*1000*8)/datatype) #you can choose 1024 insted of 1000 if you use other 2^n data
            data_n=data_n*(p)
            testmem_ip.write(0x1c,data_n)
            #start counting
            start=time.time()
            testmem_ip.write(0x00,1)
            #continue until it end 
            while testmem_ip.read(0x00)& 0x04!=0x04:
                pass
            #finish counting
            stop=time.time()
            if n==0:
                w_sum_64=w_sum_64+(stop-start)*1000
            if n==1:
                w_sum_128=w_sum_128+(stop-start)*1000
            if n==2:
                w_sum_256=w_sum_256+(stop-start)*1000
        if n==0:
            w_time_64[p-1]=w_sum_64/100
        if n==1:
            w_time_128[p-1]=w_sum_128/100
        if n==2:        
            w_time_256[p-1]=w_sum_256/100
    print(data_n)

2500000
1250000
625000


In [9]:
print(w_time_64)
print(w_time_128)
print(w_time_256)


[ 1.76483154  3.48545551  5.1997757   6.91797972  8.66033077 10.35890579
 12.07705021 13.79727602 15.51539421 17.23400116 18.9522028  20.67245245
 22.39195108 24.10716057 25.8262825  27.54600286 29.26305294 30.98481655
 32.7017045  34.42055225]
[ 1.60470009  3.1717205   4.73301888  6.2973547   7.85952806  9.42122698
 10.98455667 12.54624367 14.10959721 15.67186117 17.23342419 18.79833937
 20.35939455 21.92182302 23.48619938 25.04588842 26.60792589 28.2730484
 29.73351002 31.29555225]
[ 1.44553185  2.85339594  4.26677942  5.66838741  7.07396507  8.48154068
  9.88728523 11.29378557 12.7011919  14.10720587 15.51332951 16.91850901
 18.32675457 19.73172188 21.13835573 22.54421473 23.9499259  25.35670996
 26.77135468 28.1692481 ]


In [24]:
bit=80*1000*1000*8


n=2
#for n in range(2):
#kernel function's IP made for write from the FPGA
if n==0:
    overlay =Overlay("./design_64_wrapper.xsa")
    testmem_ip=overlay.kernel_0
    datatype=64
    w_time1_64= np.zeros(20)
if n==1:
    overlay =Overlay("./design_1_128_wrapper.xsa")
    testmem_ip=overlay.kernel_0
    datatype=128
    w_time1_128=np.zeros(20)
if n==2:
    overlay =Overlay("./design_1_256_wrapper.xsa")
    testmem_ip=overlay.kernel_0
    datatype=256
    w_time1_256=np.zeros(20)

#inizialize data
if(int(bit/datatype))>0:
    max_data=int(bit/datatype)
else:max_data=1
#FPGA
data= np.zeros(int(max_data*datatype/64))
#data buffer creation for FPGA
buf_data = allocate(int(max_data*datatype/64),np.uint64)
buf_data[:]=data[:]
buf_data.flush()
testmem_ip.write(0x10,buf_data.physical_address)

#kernel function attivation and time calculation
#starting write time calculation

#100 subtests to create an avarage time
w_sum_64=0
w_sum_128=0
w_sum_256=0
for c in range(1000):
    data_n=max_data
    testmem_ip.write(0x1c,data_n)
    #start counting
    start=time.time()
    testmem_ip.write(0x00,1)
    #continue until it end 
    while testmem_ip.read(0x00)& 0x04!=0x04:
        pass
    #finish counting
    stop=time.time()
    if n==0:
        w_sum_64=w_sum_64+(stop-start)*1000
    if n==1:
        w_sum_128=w_sum_128+(stop-start)*1000
    if n==2:
        w_sum_256=w_sum_256+(stop-start)*1000
if n==0:
    w_time1_64=w_sum_64/1000
if n==1:
    w_time1_128=w_sum_128/1000
if n==2:        
     w_time1_256=w_sum_256/1000
print(data_n)

2500000


In [25]:
print( w_time1_64)
print(w_time1_128)
print( w_time1_256)

0.06823515892028809
125.04872465133667
112.55620336532593
